In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
for_sale = pd.read_csv('chicago_for_sale_unclean.csv')
sold = pd.read_csv('chicago_sold_unclean.csv')

In [5]:
df = pd.concat([for_sale, sold]).drop(columns=['Unnamed: 0']).reset_index()
df.head(5)

,index,Type,Year built,Heating,Cooling,Parking,Price/sqft,Total Price,Address,New construction,Bedrooms,Bathrooms,Full bathrooms,neighborhood_stats,URL,Lot,Square Feet,HOA,Half bathrooms,Status
0,0,Multi Family,1921,Forced air,No Data,2 spaces,No Data,"$279,900","5931 S Campbell Ave, Chicago, IL 60629",No,6,3.0,3.0,Home values in Chicago Lawn have risen 0.6% ()...,https://www.zillow.com/homedetails/5931-S-Camp...,NaN,NaN,NaN,NaN,For Sale
1,1,Single Family,1924,Gas,Central,2 spaces,$141,"$138,970","11117 S Sawyer Ave, Chicago, IL 60655",No,3,1.0,1.0,Home values in Mount Greenwood have risen 0.6%...,https://www.zillow.com/homedetails/11117-S-Saw...,"3,789 sqft",988,NaN,NaN,For Sale
2,2,Multi Family,1930,Forced air,No Data,None,No Data,"$99,900","7151 S Green St, Chicago, IL 60621",No,6,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/7151-S-Gree...,NaN,NaN,NaN,NaN,For Sale
3,3,Single Family,1898,Gas,No Data,1 space,$27,"$39,900","620 W 60th St, Chicago, IL 60621",No,4,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/620-W-60th-...,"3,301 sqft","1,498",None,NaN,For Sale
4,4,Condo,1934,Forced air,No Data,None,$169,"$168,900","5148 N Monitor Ave APT 202, Chicago, IL 60630",No,2,1.0,1.0,Home values in Jefferson Park have risen 0.6% ...,https://www.zillow.com/homedetails/5148-N-Moni...,NaN,"1,000",$234/month,NaN,For Sale


In [16]:
# there are a few duplicate urls so we will drop those
df.drop_duplicates(subset=['URL'], inplace=True)

In [17]:
df.replace(to_replace=['None', 'No Data'], value=np.nan, inplace=True)

In [18]:
def acres_to_sqft(x):
    if isinstance(x, list):
        if 'acres' in x:
            sqft = float(x[0]) * 43560
            return round(sqft, 2)
        else:
            return float(x[0].replace(',',''))
    else:
        return x
    

df['Total Price'] = df['Total Price'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)
df['HOA'] = df['HOA'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)
df['Bedrooms'] = df['Bedrooms'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)
df['Bathrooms'] = df['Bathrooms'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)
df['Square Feet'] = df['Square Feet'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)
df['Full bathrooms'] = df['Full bathrooms'].apply(lambda x: int(x) if isinstance(x, str) else x)
df['Half bathrooms'].fillna(0, inplace=True)
df['Half bathrooms'] = df['Half bathrooms'].astype(int)
df['Year built'] = df['Year built'].apply(lambda x: int(x) if isinstance(x, str) else x)
df['Cooling'] = df.Cooling.replace('Refrigerator', 'Refrigeration')
df.Cooling.fillna('None', inplace=True)
df['Lot'] = df['Lot'].apply(lambda x: x.lower() if isinstance(x, str) else x)
df['Lot Temp'] = df['Lot'].str.split(' ')
df['Lot'] = df['Lot Temp'].apply(acres_to_sqft)
df.drop(columns=['Lot Temp', 'Price/sqft', 'index'], inplace=True)



In [19]:
pwd

'/Users/collinswestnedge/programming/Metis_Online/project_02'

In [23]:
df.dtypes

Type                   object
Year built            float64
Heating                object
Cooling                object
Parking                object
Total Price           float64
Address                object
New construction       object
Bedrooms                int64
Bathrooms             float64
Full bathrooms        float64
neighborhood_stats     object
URL                    object
Lot                   float64
Square Feet           float64
HOA                   float64
Half bathrooms          int64
Status                 object
dtype: object

In [27]:
for item in df.columns.to_list():
    print(item + ':')
    print(df[item].value_counts())
    print()

Type:
Condo                 557
Single Family         324
Multi Family           91
Multiple Occupancy     53
Townhouse              43
Apartment               1
Name: Type, dtype: int64

Year built:
2020.0    31
1920.0    31
2007.0    27
2008.0    27
2006.0    26
          ..
1870.0     1
1936.0     1
1869.0     1
1884.0     1
1896.0     1
Name: Year built, Length: 141, dtype: int64

Heating:
Forced air                         461
Gas                                221
Forced air, Gas                    186
Other                               28
Baseboard                           22
Electric                            20
Radiant                              9
Electric, Gas                        8
Forced air, Electric                 8
Other, Gas                           8
Baseboard, Forced air                5
Heat pump                            4
Forced air, Other                    3
Forced air, Electric, Gas            3
Radiant, Gas                         3
Baseboard, Gas    

In [28]:
df.to_csv('/Users/collinswestnedge/programming/Metis_Online/project_02/clean_data/chicago_full_clean.csv')